##### 1. 필요한 라이브러리 및 패키지 임포트

In [1]:
import pandas as pd
import numpy as np

# selenium의 webdriver를 사용하기 위한 import
from selenium import webdriver

# selenium으로 키를 조작하기 위한 import
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException

# 페이지 로딩을 기다리는데에 사용할 time 모듈 import
import time

import csv
from bs4 import BeautifulSoup as bs
import requests
import math

import re
import urllib.request
import json
import datetime

##### 2. 네이버 API로 네이버 스토어 url 추출

In [2]:
def link_crawler(query,start,end):
    review_url_list=[]
    
    # 네이버 developers에서 내 정보 확인
    client_id = "INPUT_client ID"
    client_secret = "INPUT_client Secret"
    
    encText = urllib.parse.quote(query)
    
    #한번에 100개 이상 수집 시 오류 발생
    idx=0
    display=10 # 한번에 가져오는 페이지 개수
    start=start # 시작 게시물 번호
    end=end # 종료 게시물 번호
    
    
    
    for st_idx in range(start,end,display):
        url = "https://openapi.naver.com/v1/search/shop?query=" + encText + "&display=" + str(display) + "&start=" + str(st_idx) # 네이버 블로그 검색 API 호출 URL 생성
        
        # 요청 객체 생성 및 헤더 추가
        request = urllib.request.Request(url)
        request.add_header("X-Naver-Client-Id",client_id)
        request.add_header("X-Naver-Client-Secret",client_secret)

        # API 요청 및 응답 확인
        response = urllib.request.urlopen(request)
        rescode = response.getcode()
    
        if(rescode==200): # 정상 반응
            response_body = response.read()
            data=response_body.decode('utf-8')
            data = json.loads(data)
            links = [item['link'] for item in data['items']] # 링크추출
            
            for link in links:
                # 브랜드 스토어 링크만 리스트에 담기
                if 'smartstore' in link: 
                    review_url_list.append(link)
            
    
        else: # 예외처리
            print("Error Code:" + rescode) 
            
    return(review_url_list)

In [ ]:
review_url_list = link_crawler("INPUT_검색 키워드", 1, n)
print(len(review_url_list))
print(review_url_list)

##### 3. 상품 리뷰 크롤링

In [ ]:
reviews_list = []

for url in review_url_list: 
    driver = webdriver.Chrome()
    driver.get(url)
    time.sleep(random.uniform(0.5,1.3)) # 요청시간 랜덤 조정으로 크롤링 차단 방지

    driver.find_element(By.XPATH, '//*[@id="content"]/div/div[2]/div[1]/div[2]/div[2]/a').click() # 상품 리뷰 페이지 클릭
    time.sleep(random.uniform(0.5,1.3)) 
    
    # 네이버 스토어 리뷰의 경우 한 페이지당 리뷰 20개
    page = 1 
    max_page = 500 # 크롤링 할  페이지 개수 설정
    review_list = []

    while page <= max_page:
        try: # 페이지 소스 가져오기
            soup = bs(driver.page_source, 'lxml')
            ul_list = soup.find('ul', {'class': '_2ms2i3dD92'})
            reviews = ul_list.find_all('li') # 한 페이지에 있는 리뷰 모두 추출
        
            for review in reviews:
                review_div = review.find('div', {'class': '_3z6gI4oI6l'}) # 리뷰 텍스트만 추출
                review_star = review.find('div', {'class': '_2V6vMO_iLm'}) # 평점(별점) 추출
                
                if review_div:
                    text = review_div.get_text()
                    text = text.replace('더보기', '').strip() #'더보기' 제거 (전체 텍스트를 크롤링 해도 끝에 더보기가 뜸)
                    star = review_star.get_text()
                    star = star.replace('평점','').strip() # '평점'제거하고 숫자만
                    review_list.append([text,star])
        
        except Exception as e: # 예외처리
            print('예상치 못하게 종료')
            break

        try: # 다음 페이지 이동
            if (page % 10) == 0: # 현재 페이지가 10,20.. 단위일 때
                next_page_button = driver.find_element(By.XPATH, '//*[@id="REVIEW"]/div[1]/div[3]/div[2]/div/div/a[12]') # 페이지 넘김 버튼
            else: 
                next_page_button = driver.find_element(By.XPATH, f'//*[@id="REVIEW"]/div[1]/div[3]/div[2]/div/div/a[{(page % 10) + 2}]') # 다음 페이지 버튼

            try:
                next_page_button.click() # 페이지 이동 버튼 클릭
                page += 1
                time.sleep(1)

            except NoSuchElementException: # 다음 페이지 버튼이 없을 때 (마지막 페이지인 경우)
                print("다음 페이지 버튼을 찾을 수 없습니다. 마지막 페이지입니다.")
                break

            except ElementNotInteractableException:  # 예외처리
                print("요소를 클릭할 수 없습니다. 페이지를 종료합니다.")
                break

        except Exception as e: # 예외처리
            print("다음 페이지 버튼을 찾을 수 없습니다. 마지막 페이지입니다.")
            break

    reviews_list.append(review_list) 
    driver.quit()

print("크롤링 완료")

##### 4. 데이터 저장

In [16]:
import itertools
reviews_list_merge = list(itertools.chain(*reviews_list)) #각 상품 리뷰를 하나의 리스트로 합치기

reviews_df = pd.DataFrame(reviews_list_merge)
reviews_df.to_csv('NaverProductReview.csv',index=False,encoding='utf-8-sig')